# Puzzle 7

“Oh that tapestry, with the colorful toucan on it! I’ll tell you what happened to it.

“One day, I was at Noah’s Market, and I was just about to leave when someone behind me said ‘Miss! You dropped something!’

“Well I turned around and sure enough this cute guy was holding something I had bought. He said ‘I got almost exactly the same thing!’ We laughed about it and wound up swapping items because he had wanted the color I got. We had a moment when our eyes met and my heart stopped for a second. I asked him to get some food with me and we spent the rest of the day together.

“Before long I moved into his place. It didn’t last long though, as I soon discovered this man was not the gentleman I thought he was. I moved out only a few months later, late at night and in quite a hurry.

“I realized the next day that I’d left that tapestry hanging on his wall. But the tapestry had come to represent our relationship, and I wanted nothing more to do with him, so I let it go. For all I know, he still has it.”

Can you figure out her ex-boyfriend’s phone number?

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import os
from nameparser import HumanName
import datetime as dt

In [2]:
dir = "/Users/jattenberg/Downloads/noahs-csv/"
files = os.listdir(dir)
names = [n.split(".")[0] for n in files]
dataframe_arr = [pd.read_csv(dir+f) for f in files]

dfs = dict(zip(names, dataframe_arr))

In [14]:
products = dfs["noahs-products"]
order_items = dfs["noahs-orders_items"]
orders = dfs["noahs-orders"]
customers = dfs["noahs-customers"]

orders["ordered_date"] = pd.to_datetime(orders.ordered).dt.strftime('%d_%m_%Y')
# products with a color use parens at the end of the description
products["colorless_desc"] = products.desc.str.replace("\(.*\)", "", regex=True) 

fully_denormalized = (
    customers.merge(orders, on="customerid")
        .merge(order_items, on="orderid")
        .merge(products, on="sku")
)

In [4]:
person_from_last_time = customers.set_index("customerid").loc[[8342]].reset_index()
person_from_last_time

,customerid,name,address,citystatezip,birthdate,phone
0,8342,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316


In [12]:
products_she_got = person_from_last_time.merge(fully_denormalized)[["desc", "colorless_desc", "ordered_date"]]
products_she_got[products_she_got.desc != products_she_got.colorless_desc]

,desc,colorless_desc,ordered_date
4,Noah's Poster (white),Noah's Poster,09_09_2019
9,Electric Machine (amber),Electric Machine,01_06_2019
10,Manual Mixer (orange),Manual Mixer,18_08_2019
11,Electric Crockpot (purple),Electric Crockpot,17_04_2021
16,Electric Toaster (blue),Electric Toaster,18_08_2019
19,Handmade Toaster (blue),Handmade Toaster,28_05_2017
23,Noah's Jewelry (orange),Noah's Jewelry,26_07_2022
24,Noah's Jewelry (yellow),Noah's Jewelry,17_04_2021


### Want: orders with the same colorless description and order date, with different actual desc

In [10]:
possible_matches = products_she_got.merge(fully_denormalized, on=["ordered_date", "colorless_desc"])
possible_matches[possible_matches.desc_x != possible_matches.desc_y]

,desc_x,colorless_desc,ordered_date,customerid,name,address,citystatezip,birthdate,phone,orderid,ordered,shipped,items,total,sku,qty,unit_price,desc_y,wholesale_cost
11,Electric Machine (amber),Electric Machine,01_06_2019,8835,Jonathan Adams,644 Targee St,"Staten Island, NY 10304",1975-08-26,315-618-5263,86372,2019-06-01 12:50:25,2019-06-01 12:50:25,NaN,34.82,HOM7757,1,34.82,Electric Machine (purple),29.43
20,Electric Toaster (blue),Electric Toaster,18_08_2019,4236,Melvin Rodriguez III,228A E 115th St,"Manhattan, NY 10029",1970-11-22,914-698-1257,94026,2019-08-18 12:47:14,2019-08-18 12:47:14,NaN,29.91,HOM5134,1,29.91,Electric Toaster (mauve),24.02
28,Noah's Jewelry (orange),Noah's Jewelry,26_07_2022,5911,Beth Mack,2193 White Plains Rd,"Bronx, NY 10462",1959-12-12,315-444-9913,200976,2022-07-26 12:05:26,2022-07-26 20:00:00,NaN,91.68,COL9689,1,82.80,Noah's Jewelry (azure),62.01
30,Noah's Jewelry (yellow),Noah's Jewelry,17_04_2021,8341,James Anderson,1432A Ogden Ave,"Bronx, NY 10452",1962-07-29,516-689-9580,154856,2021-04-17 15:54:57,2021-04-17 16:30:00,NaN,72.71,COL8453,1,60.48,Noah's Jewelry (blue),51.90
31,Noah's Jewelry (yellow),Noah's Jewelry,17_04_2021,4274,Timothy Jones,755 7th Ave,"Manhattan, NY 10019",1989-11-06,716-691-6727,154816,2021-04-17 11:14:30,2021-04-17 11:14:30,NaN,196.73,COL3050,1,73.91,Noah's Jewelry (green),59.54
